In [2]:
import pandas as pd
import os,re
import numpy as np

In [3]:
parsed = os.listdir('Data')
parsed = [i for i in parsed if '2020' in i]

In [4]:
codes = {
    2011:{'MT':130,'CN':123,'CH':119,'LC':126},
    2012:{'CN':153,'CH':154,'LC':155,'MT':156},
    2013:{'CN':188,'CH':187,'LC':189,'MT':190},
    2015:{'CN':277,'CH':273,'LC':280,'MT':284},
    2016:{'CN':307,'CH':308,'LC':309,'MT':310},
    2017:{'CN':407,'CH':408,'LC':409,'MT':410},
    2018:{'CN':463,'CH':464,'LC':465,'MT':466},
    2019:{'CN':519,'CH':520,'LC':521,'MT':522},
    2020:{'CN':604,'CH':574,'LC':584,'MT':594},
    2021:{'CN':916,'CH':886,'LC':896,'MT':906},
    2022:{'CN':1092,'CH':1062,'LC':1072,'MT':1082},
    2023:{'CN':0,'CH':0,'LC':0,'MT':0}
}

In [4]:
#remove dumb parsing artifacts from 2023
year='2023'
base_path = "./Data/"
enems = os.listdir(base_path)
for file in enems:
    if year in file:
        df = pd.read_csv('Data/'+file)
        for column in df:
            df[column] = df[column].str.replace(r' -(?=[A-Za-z])','-',regex=True)
            df[column] = df[column].str.replace(r'(?<=[A-Za-z])- ','-',regex=True)
            df[column] = df[column].str.replace(r' -(?=[\d])','-',regex=True)
            df[column] = df[column].str.replace(r'(?<=[\d])- ','-',regex=True)
        #fix maths in the answers
        for col in ['A','B','C','D','E','body']:
            df[col] = df[col].apply(lambda text: re.sub(' ?,? ?[Ff]echa parêntese ?,?',')',re.sub(' ?,? ?[Aa]bre parêntese ?,? ?',' (',text)) if type(text)==str else text)
        df.to_csv('Data/'+file,index=False)
        display(df.iloc[-3]['body'])

'Do século 16 em diante, pelo menos nas classes mais altas, o garfo passou a ser usado como utensílio para comer, chegando através da Itália primeiramente à França e, em seguida, à Inglaterra e à Alemanha, depois de ter servido, durante algum tempo, apenas para retirar alimentos sólidos da travessa. Henrique Terceiro introduziu-o na França, trazendo-o provavelmente de Veneza. Seus cortesãos não foram pouco ridicularizados por essa maneira “afetada” de comer e, no princípio, não eram muito hábeis no uso do utensílio: pelo menos se dizia que metade da comida caía do garfo no caminho do prato à boca. Em data tão recente como o século 17, o garfo era ainda basicamente artigo de luxo, geralmente feito de prata ou ouro.  O processo social relatado indica a formação de uma etiqueta que tem como princípio a'

'De quem é esta língua?  José Eduardo Agualusa  Uma pequena editora brasileira, a Urutau, acaba de lançar em Lisboa uma “antologia antirracista de poetas estrangeiros em Portugal”, com o título Volta para a tua terra . O livro denuncia as diversas formas de racismo a que os imigrantes estão sujeitos. Alguns dos poetas brasileiros antologiados queixam-se do desdém com que um grande número de portugueses acolhe o português brasileiro. É uma queixa frequente. “Aqui em Portugal eles dizem / – eles dizem – / que nosso português é errado, que nós não falamos português”, escreve a poetisa paulista Maria Giulia Pinheiro, para concluir: “Se a sua linguagem, a lusitana, / ainda conserva a palavra da opressão / ela não é a mais bonita do mundo./ Ela é uma das mais violentas”.  O texto de Agualusa tematiza o preconceito em relação ao português brasileiro. Com base no trecho citado pelo autor, infere-se que esse preconceito se deve'

'Em uma indústria, o controle da dureza da água é importante quando ela é utilizada em caldeiras, uma vez que sais pouco solúveis, formados a partir de sulfatos e carbonatos, podem acumular-se no interior das tubulações, causando obstruções. Para avaliar a água utilizada nessa indústria, foram realizados testes de qualidade que consideraram os parâmetros apresentados no quadro .  Descrição do quadro: O quadro indica os parâmetros medidos para testes de 1 a 5:  Teste 1: cálcio;  Teste 2: cloreto;  Teste 3: turbidez;  Teste 4: coliformes totais;  Teste 5: sólidos sedimentáveis.  (Fim da descrição)   Qual teste deve ser considerado para controlar a formação desse tipo de obstrução de tubulações?'

'O calendário maia apresenta duas contagens simultâneas de anos, o chamado ano Tzolkim, composto por 260 dias e que determinava o calendário religioso, e o ano Haab, composto por 365 dias e que determinava o calendário agrícola. Um historiador encontrou evidências de que gerações de uma mesma família governaram certa comunidade maia pelo período de 20 ciclos, sendo cada ciclo formado por 52 anos Haab.  De acordo com as informações fornecidas, durante quantos anos Tzolkim aquela comunidade maia foi governada por tal família?'

In [5]:
dfs = {}


def match_gab_to_df(df,gab):
    answers=[]
    languages=[]
    used = set()
    for line in df.to_dict(orient='records'):
        if line['test']=='redação':
            answers.append('')
            languages.append('portuguese')
            continue
        possible_gabarito = gab[gab['question']==line['question_number']]
        gabarito = possible_gabarito.iloc[0]
        
        if (str(gabarito['question'])+str(gabarito['test'])) in used:
            gabarito = possible_gabarito.iloc[1]
            
        answers.append(gabarito['answer'])
        languages.append(gabarito['test'] if not pd.isna(gabarito['test']) else 'portuguese')
        used.add(str(gabarito['question'])+str(gabarito['test']))
    return answers,languages


for prova in parsed:
    print(prova)
    df = pd.read_csv('Data/'+prova)
    print(len(df))
    year = int(re.search("\d\d\d\d",prova).group())
    df['question'] = df['question'].apply(lambda question: question.upper())
    df['year'] = year
    df['question_number'] = df['question'].apply(lambda question: int(re.search("[\d]+",question).group()) if re.search("[\d]+",question) else np.nan)
    df['test']='redação'
    if 'D1_' in prova or 'ia1' in prova or 'DIA_1' in prova:
        if year < 2017:
            df.loc[df['question_number']<=45,'test'] = 'CH'
            df.loc[df['question_number']>45,'test'] = 'CN'
        else:
            df.loc[df['question_number']<=45,'test'] = 'LC'
            df.loc[df['question_number']>45,'test'] = 'CH'
    else:
        if year < 2017:
            df.loc[df['question_number']<=135,'test'] = 'LC'
            df.loc[df['question_number']>135,'test'] = 'MT'
        else:
            df.loc[df['question_number']<=135,'test'] = 'CN'
            df.loc[df['question_number']>135,'test'] = 'MT'
    #add gabarito data
    if year != 2023:
        gab = pd.read_csv('GabaritosCsv/'+prova.replace('CAD','GAB')).sort_values(by=['question','test'])
    else:
        gab = pd.read_csv('GabaritosCsv/'+prova.replace('_CH','').replace('_LC','').replace('_CN','').replace('_MT',''))
    df['answer'],df['language'] = match_gab_to_df(df,gab)
    #we want to sort dataframe in the following order: english -> spanish -> portuguese then question orders
    #this is useful for the adding of microdata
    df['sort_index'] = df['language'].apply(lambda lang: 0 if lang == 'english' else 1 if lang=='spanish' else 2)
    df = df.sort_values(by=['sort_index','question_number']).drop(columns=['sort_index'])
    
    #fix maths in the answers
    for col in ['A','B','C','D','E','body']:
        df[col] = df[col].apply(lambda text: re.sub(' ?,? ?[Ff]echa parêntese ?,?',')',re.sub(' ?,? ?[Aa]bre parêntese ?,? ?',' (',text)) if type(text)==str else text)
    
    #add microdata data
    if year != 2023:
        important_columns = ['CO_PROVA','NU_PARAM_A','NU_PARAM_B','NU_PARAM_C','TP_LINGUA','CO_POSICAO','CO_HABILIDADE','TX_GABARITO']
        microdata = pd.read_csv(f'microdados/ITENS_PROVA_{year}.csv',sep=';',usecols=important_columns, encoding='latin-1')
        microdata.loc[microdata['TP_LINGUA'].isna(),'TP_LINGUA']='portuguese'
        microdata.loc[microdata['TP_LINGUA']==0,'TP_LINGUA']='english'
        microdata.loc[microdata['TP_LINGUA']==1,'TP_LINGUA']='spanish'
    

#     #divide tests
    for test in df['test'].drop_duplicates().tolist():
#         print(prova+' '+ test)
        one_subject_test = df[df['test']==test].copy().reset_index(drop=True)
        if test == 'redação':
            one_subject_test.to_csv(f'ProvasComMicrodados/ENEM_{year}_{test}.csv')
            continue
        else:
            CO_PROVA = codes[year][test]
#             specific_microdata = microdata[microdata['CO_PROVA']==CO_PROVA].copy()
#             one_subject_test['CO_PROVA'] = CO_PROVA

#             try:
#                 output = pd.merge(one_subject_test,specific_microdata,left_on=['question_number','language'],right_on=['CO_POSICAO','TP_LINGUA'])
#                 if output.empty:
#                     raise Exception(f'Failed matching')
#             except:
#                 one_subject_test['match'] = list(one_subject_test.index+1)
#                 output = pd.merge(one_subject_test,specific_microdata,left_on=['match'],right_on=['CO_POSICAO']).drop(columns=['match'])
#                 if not (output['TX_GABARITO']!=output['answer']).sum()==0:
#                     print(f'Gabarito and answer should be the same for test {prova} {test}')
#                     #CO_PROVA = str(CO_PROVA) + 'MAYBEBROKENDONOTUSE'
#             if output.empty:
#                 print('Output empty??')
#             if len(output)!=45 and len(output)!=50:
#                 raise Exception('Inconsistent Output')
#             output.drop(columns=['CO_PROVA_x','TP_LINGUA','CO_POSICAO'],inplace=True)
#             output.rename(columns={'CO_PROVA_y':'CO_PROVA'},inplace=True)
#             output['question_number'] = output['question_number'].astype(int)
        one_subject_test['question_number']=one_subject_test['question_number'].astype(int)
        if len(one_subject_test)!=45 and len(one_subject_test)!=50:
            print(prova,len(one_subject_test),test)
        one_subject_test.to_csv(f'ProvasComMicrodados/ENEM_{year}_{test}_CO_PROVA_{CO_PROVA}.csv',index=False,encoding='utf-8')
    dfs[prova] = df

2020_PV_CAD_impresso_D1_CD9_laranja.csv
96
2020_PV_CAD_impresso_D2_CD11_laranja.csv
90


In [7]:
#remove dumb parsing artifacts from specific provas
year=2022
base_path = "./ProvasComMicrodados/"
enems = os.listdir(base_path)
df = pd.concat([pd.read_csv(base_path+file) for file in enems ]).sort_values('question_number').reset_index(drop=True)
for file in enems:
    if '2022' in file and 'redação' not in file:
        df = pd.read_csv(base_path+file)
        df['A'] = df['A'].str.lstrip('A').str.strip(' ')
        print(df['A'].isna().sum())
        df['B'] = df['B'].str.lstrip('B').str.strip(' ')
        print(df['B'].isna().sum())
        df['C'] = df['C'].str.lstrip('C').str.strip(' ')
        print(df['C'].isna().sum())
        df['D'] = df['D'].str.lstrip('D').str.strip(' ')
        print(df['D'].isna().sum())
        df['E'] = df['E'].str.lstrip('E').str.strip(' ')
        print(df['E'].isna().sum())
        df.to_csv(base_path+file,encoding='utf-8',index=False)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [35]:
prova = 'ia2_caderno6_CAD_cinza_ledor_2013.csv'
df = pd.read_csv('data/ia2_caderno6_CAD_cinza_ledor_2013.csv ',encoding='utf-8')
year=2013
df['question'] = df['question'].apply(lambda question: question.upper())
df['year'] = year
df['question_number'] = df['question'].apply(lambda question: int(re.search("[\d]+",question).group()) if re.search("[\d]+",question) else np.nan)
df['test']='redação'
if 'D1_' in prova or 'ia1' in prova or 'DIA_1' in prova:
    if year < 2017:
        df.loc[df['question_number']<=45,'test'] = 'CH'
        df.loc[df['question_number']>45,'test'] = 'CN'
    else:
        df.loc[df['question_number']<=45,'test'] = 'LC'
        df.loc[df['question_number']>45,'test'] = 'CH'
else:
    if year < 2017:
        df.loc[df['question_number']<=135,'test'] = 'LC'
        df.loc[df['question_number']>135,'test'] = 'MT'
    else:
        df.loc[df['question_number']<=135,'test'] = 'CN'
        df.loc[df['question_number']>135,'test'] = 'MT'